In [1]:
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
orig_seq = 'abcdefghijklmnopqrstuvwxyz'

mutrate = 0.05
insrate = 0.01
delrate = 0.01

nbseq = 10000

In [5]:
def generate_sequences(orig_seq, nbseq,
                       mutrate=0.05, 
                       insrate=0.01, 
                       delrate=0.01,
                       frontcutlen=0.1,
                       endcutlen=0.1):
    seqelemset = set(orig_seq)

    for _ in range(nbseq):
        thisseq = orig_seq

        # mutation
        for pos in range(len(thisseq)):
            if np.random.uniform() < mutrate:
                picked_char = np.random.choice(list(seqelemset - {thisseq[pos]}))
                thisseq = thisseq[:pos]+picked_char+thisseq[(pos+1):]

        # insertion
        inspts = [pos for pos in range(len(thisseq)) if np.random.uniform() < insrate]
        for idx, inspt in enumerate(inspts):
            picked_char = np.random.choice(list(seqelemset))
            thisseq = thisseq[:(inspt+idx)] + picked_char + thisseq[(inspt+idx):]

        # deletion
        delpts = [pos for pos in range(len(thisseq)) if np.random.uniform() < delrate]
        for idx, delpt in enumerate(delpts):
            thisseq = thisseq[:(delpt-idx)] + thisseq[(delpt-idx+1):]

        # strip front and end
        cutfront = np.random.poisson(frontcutlen)
        cutend = np.random.poisson(endcutlen)
        thisseq = thisseq[cutfront:(len(thisseq)-cutend)]

        yield thisseq

In [6]:
[seq for seq in generate_sequences(orig_seq, 10)]

['abcdefghijklmnopqbrstudwny',
 'abcdefghijkgmnopqzstuvwxyz',
 'abcdefghiiklmnopqrstuvwxyz',
 'abcdefjhijklmnopkruituvwxyz',
 'abcdefghijklmnopqrstuvwxy',
 'pbcdefghijklmnopqmstuvwxyz',
 'abcdefghijklmnopsrstuvwxyz',
 'bcdefghijklmnopqrstuvwxyz',
 'abcdefghijkmmnopqrstuvwxyz',
 'cbcdefghitlmnopqrstuvwxyz']

In [8]:
# from keras.models import Sequential
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras import Input
from keras.engine import Model
from keras.utils import to_categorical

In [13]:
chartoidx_dict = {character: index for index, character in enumerate(orig_seq)}

In [24]:
def charseq_to_vectors(charseq, chartoidx_dict):
    onehot = [chartoidx_dict[c] for c in charseq]
    return to_categorical(onehot, num_classes=len(chartoidx_dict))

In [27]:
charseq_to_vectors('origseq', chartoidx_dict).shape

(7, 26)

In [ ]:
# specifying the network
input_seq = Input(shape=(None, len(chartoidx_dict)))
encoded = LSTM(100, activation='relu')(input_seq)
encoded = RepeatVector()(encoded)
